# Design Patterns

## Metaclasses

Class object is a instance of class "type". It is a default metaclass.
Metaclass is responsible how class instantiates and operates.  

In [3]:
class MyType(type):
    pass

class MyClass(metaclass=MyType):
    pass

In [4]:
a = MyClass()
type(a)

__main__.MyClass

In [6]:
type(MyClass)

__main__.MyType

In [7]:
type(MyType)

type

## Implementing Singleton using metaclasses

In [11]:
class Singleton(type):
    instance = None
    def __call__(cls, *args, **kw):
        if not cls.instance:
             cls.instance = super(Singleton, cls).__call__(*args, **kw)
        return cls.instance
    
class UniqIbject(metaclass=Singleton):
    pass

In [12]:
a = UniqIbject()
b = UniqIbject()
a is b

True

Singleton class can have only one instance.

## Abstract Base Classes

what is the best way to test that an object exposes a given interface? Abstract Base Classes provide a way to attach to a concrete class a virtual class with the only purpose of signaling a promised behaviour to anyone inspecting it with isinstance() or issubclass(). 

In [14]:
from abc import ABCMeta

class MyABC(metaclass=ABCMeta):
    pass

MyABC.register(list)

In [15]:
issubclass(list, MyABC)

True

In [17]:
isinstance([], MyABC)

True

## Usefull decorators for OOP

In [61]:
import random
import abc


class Pet(metaclass=abc.ABCMeta):
    
    @abc.abstractproperty
    def name(self):
        pass

class Cat(Pet):
    char_voice = 'meow'
    
    def __init__(self, name):
        self._name = name
    
    @property
    def name(self):
        print('getter is called')
        return self._name
    
    @name.setter
    def name(self, x):
        print('setter is called')
        self._name = x
    
    def is_alive(self):
        return True
    
    @classmethod
    def voice(cls):
        return cls.char_voice
    
class SchredingerCat(Cat):
    prob_threshold = 0.5
    
    def __init__(self, name):
        super().__init__(name)
        self.experiment_result = random.random() >= self.prob_threshold

    @property
    def name(self):
        return 'schredinger ' + self._name
    
    def is_alive(self):
        return self.experiment_result
    
    def voice(self):
        return '%s says %s with prob %.1f' % (super().get_name(), self.char_voice, self.prob_threshold)

In [62]:
c = Cat('a')
c.name

getter is called


'a'

In [63]:
c.name = 'b'

setter is called


In [64]:
c = SchredingerCat('a')
c.name

'schredinger a'

In [65]:
c = Pet()

TypeError: Can't instantiate abstract class Pet with abstract methods name

@property decorator provides some sort of incapsulation.  
@abc.abstract... decorator provide pure abstract methods. Class cannot be instantiated if it doesn't override all abstract methods.

## Abstract Fabric

Provide an interface for creating families of related or dependent objects without specifying their concrete classes

In [71]:
from abc import ABCMeta, abstractmethod

class Ball:
    def __init__(self, size):
        self._size = size
        
    @property
    def size(self):
        return self._size

class Dog(metaclass=abc.ABCMeta):
    @abstractmethod
    def fetch(self, ball, dist):
        pass
    
class Cat(metaclass=abc.ABCMeta):
    @abstractmethod
    def play(self, ball):
        pass
    
class MastiffDog(Dog):
    def fetch(self, ball, dist):
        return dist < 1000
        
        
class DachshundDog(Dog):
    def fetch(self, ball, dist):
        return ball.size < 10 and dist < 100 


class PersianCat(Cat):
    def play(self, ball):
        return ball.size < 10
    
class TigerCat(Cat):
    def play(self, ball):
        return True    

In [79]:
class AbstractFactory(metaclass=abc.ABCMeta):
    @abstractmethod
    def get_pet(self):
        pass
    
    
class DogFactory(AbstractFactory):
    def get_pet(self, atype):
        if atype == None:
            return None

        if atype == "small":
            return DachshundDog()
        elif atype == "big":
            return MastiffDog()

        return None
    

class CatFactory(AbstractFactory):
    def get_pet(self, atype):
        if atype == None:
            return None

        if atype == "small":
            return PersianCat()
        elif atype == "big":
            return TigerCat()

        return None

In [80]:
small_ball = Ball(1)
big_ball = Ball(20)
    

factory = DogFactory()
print(factory.get_pet('small').fetch(small_ball, 1))
print(factory.get_pet('small').fetch(small_ball, 1000))
print(factory.get_pet('big').fetch(big_ball, 1))
print(factory.get_pet('big').fetch(big_ball, 1000))

True
False
True
False


In [82]:

factory = CatFactory()
print(factory.get_pet('small').play(small_ball))
print(factory.get_pet('small').play(big_ball))
print(factory.get_pet('big').play(small_ball))
print(factory.get_pet('big').play(big_ball))

True
False
True
True


## Decorator

In [88]:
from functools import wraps


def exclamation(function):
    """Defines the decorator"""
 
    #This makes the decorator transparent (name and docs)
    @wraps(function)
 
    #Define the inner function
    def decorator():
        #Grab the return value of the function being decorated
        ret = function() 
 
        #Add new functionality
        return "!!! " + ret + " !!!"
 
    return decorator


@exclamation
def foo():
    """sample docstr"""
    
    return "foo"

!!! foo !!!
foo
sample docstr


In [89]:
print(foo())

!!! foo !!!


In [90]:
print(foo.__name__)

foo


In [91]:
print(foo.__doc__)

sample docstr


## Adapter

Adapt different class interfaces into expected one.

In [99]:
class Scotland:
 
	def speak_scottish(self):
		return "madainn mhath"

class England:
	def speak_english(self):
		return "Good Morning"	
    
    
class LanguageAdapter:
    def __init__(self, model, **adapted_method):
        self._model = model
        self.__dict__.update(adapted_method)
 
    def __getattr__(self, attr):
        return getattr(self._model, attr)
 
a = Scotland()
b =England()
 
ax = LanguageAdapter(a, greetings=a.speak_scottish)
bx = LanguageAdapter(b, greetings=b.speak_english)

ax.greetings(), bx.greetings()

('madainn mhath', 'Good Morning')

## Observer

A software design pattern in which an object, called the subject, maintains a list of its dependents, called observers, and notifies them automatically of any state changes, usually by calling one of their methods. It is mainly used to implement distributed event handling systems.

In [114]:
class Observer(object): 
 
    def __init__(self):
        self._observers = []

    def attach(self, observer):
        if observer not in self._observers:
            self._observers.append(observer) 

    def detach(self, observer): 
        try:
            self._observers.remove(observer)
        except ValueError:
            pass

    def notify(self, modifier=None):
        for observer in self._observers: 
            if modifier != observer:  
                observer.update(self) 
                
 
class Parking(Observer): 
 
    def __init__(self, name=""):
        super().__init__()
        self._name = name 
        self._n_slots = 0 

    @property
    def n_slots(self):
        return self._n_slots

    @n_slots.setter
    def n_slots(self, n):
        self._n_slots = n
        self.notify()
        
 
class ParkingDisplay:
    def update(self, x):
        print("Parking: {} has {} slots available".format(x._name, x.n_slots))
 

a = Parking("A")
 
#Let's create our observers
v1 = ParkingDisplay()
v2 = ParkingDisplay()
 
#Let's attach our observers to the first core
a.attach(v1)
a.attach(v2)
 
#Let's change the temperature of our first core
a.n_slots = 20
a.n_slots = 2

Parking: A has 20 slots available
Parking: A has 20 slots available
Parking: A has 2 slots available
Parking: A has 2 slots available


## Strategy

Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it

In [118]:
import types

 
class SortStrategy:
 
    def __init__(self, func):
        self.func = func
 
    def execute(self, a): 
        print(self.func.__name__ + ' used')
        return self.func(a)
 
def ascending(a):
    return sorted(a)
 
# Replacement method 2    
def descending(a):
	return sorted(a, reverse=True)
 

a = [1 , 5, 8 , 4, -1]
s1 = SortStrategy(ascending)
s1.execute(a)

ascending used


[-1, 1, 4, 5, 8]

In [119]:
s2 = SortStrategy(descending)
s2.execute(a)

descending used


[8, 5, 4, 1, -1]